In [45]:
!rm /content/News_CSVs

In [46]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!ln -s "/content/drive/MyDrive/News_CSVs"

Mounted at /content/drive


In [47]:
!pip install keras


In [48]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


# Importing data and exploration

In [49]:
# Import packages
import pandas as pd
import numpy as np
import nltk, string,re
from string import punctuation
from nltk.corpus import stopwords
from sklearn import metrics
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col, to_date, rand, lower
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [50]:
#Load the dataset into a PySpark DataFrame
fake_df = spark.read.csv("/content/News_CSVs/fake.csv", header=True, inferSchema=True)
fake_df.show()


+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|
| Papa John’s Foun...|A centerpiece of ...|   News|December 21, 2017|
| WATCH: Paul Ryan..

In [51]:
fake_df.count()

23489

In [52]:
#Load the dataset into a PySpark DataFrame
true_df = spark.read.csv("/content/News_CSVs/true.csv", header=True, inferSchema=True)
true_df.show()


+--------------------+--------------------+--------------------+------------------+
|               title|                text|             subject|              date|
+--------------------+--------------------+--------------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Factbox: Trump on...|The following sta...|        politicsNews|December 29, 2017 |
|Trump on Twitter ...|The following sta...|        politicsNews|December 29,

In [53]:
true_df.count()

21417

# Data cleaning

In [54]:
# Display the schema of the fake dataframe
print("Fake DataFrame Schema:")
fake_df.printSchema()

# Display the schema of the true dataframe
print("\nReal DataFrame Schema:")
true_df.printSchema()

Fake DataFrame Schema:
root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)


Real DataFrame Schema:
root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)



In [55]:
fake_df = fake_df.dropna(how='any')

In [56]:
fake_df.count()

23481

In [57]:
true_df = true_df.dropna(how='any')

In [58]:
true_df.count()

21417

In [59]:
# Add a new column named 'label' with value 'fake' to the fake dataframe
fake_df = fake_df.withColumn('label', lit(0))

# Add a new column named 'label' with value 'true' to the real dataframe
true_df = true_df.withColumn('label', lit(1))

In [60]:
# Merge datasets
news_data_df = fake_df.unionByName(true_df)

# show the concatenated dataframe
news_data_df.show()

+--------------------+--------------------+-------+-----------------+-----+
|               title|                text|subject|             date|label|
+--------------------+--------------------+-------+-----------------+-----+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|    0|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|    0|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|    0|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|    0|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|    0|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|    0|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|    0|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|    0|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|    0|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|    0|
| Papa John’

In [61]:
#Random Shuffling the dataframe
news_data_df = news_data_df.orderBy(rand())
news_data_df.show()


+--------------------+--------------------+--------------------+--------------------+-----+
|               title|                text|             subject|                date|label|
+--------------------+--------------------+--------------------+--------------------+-----+
|Meet Leftist Vogu...|A fashion editor ...|           left-news|        Aug 31, 2017|    0|
|Netanyahu muzzles...|JERUSALEM (Reuter...|           worldnews| September 25, 2017 |    1|
|China and India a...|BEIJING (Reuters)...|           worldnews|  September 5, 2017 |    1|
|HELL-BENT ON A CO...|Which begs the qu...|     Government News|         Jun 8, 2015|    0|
|THE NEW SWEDEN: R...|The consequences ...|     Government News|         Jan 2, 2016|    0|
|Pelosi re-elected...|WASHINGTON (Reute...|        politicsNews|  November 30, 2016 |    1|
|Trump dismisses B...|WASHINGTON (Reute...|        politicsNews|     August 2, 2016 |    1|
|SHOCKING NEW MOVI...|Dennis Michael Ly...|     Government News|         Jan 1, 

In [62]:
#combine two columns title and text
from pyspark.sql.functions import concat_ws
news_data_df = news_data_df.withColumn('news', concat_ws(" ", news_data_df.title, news_data_df.text))

In [63]:
news_data_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- label: integer (nullable = false)
 |-- news: string (nullable = false)



In [64]:
#Drop extra column
news_data_df = news_data_df.drop('title').drop('text').drop('date').drop('subject')
news_data_df.show()

+-----+--------------------+
|label|                news|
+-----+--------------------+
|    0|Meet Leftist Vogu...|
|    1|Netanyahu muzzles...|
|    1|China and India a...|
|    0|HELL-BENT ON A CO...|
|    0|THE NEW SWEDEN: R...|
|    1|Pelosi re-elected...|
|    1|Trump dismisses B...|
|    0|SHOCKING NEW MOVI...|
|    1|About 500 French ...|
|    0| State Department...|
|    0|John McCain Throw...|
|    0|FORBES LIST OF “T...|
|    1|Hurricane Irma th...|
|    0|WAS IRANIAN BUSIN...|
|    0|FROM THE RELIGION...|
|    1|Pakistan bars a m...|
|    0|GOP PRESIDENTIAL ...|
|    0|Ep #10: Patrick H...|
|    0|JUDICIARY MEMBER:...|
|    0|‘YOU’RE A SEXIST ...|
+-----+--------------------+
only showing top 20 rows



In [65]:
# convert in to lowercase
news_data_df = news_data_df.withColumn('news', lower(col('news')))
news_data_df.show()


+-----+--------------------+
|label|                news|
+-----+--------------------+
|    0|meet leftist vogu...|
|    1|netanyahu muzzles...|
|    1|china and india a...|
|    0|hell-bent on a co...|
|    0|the new sweden: r...|
|    1|pelosi re-elected...|
|    1|trump dismisses b...|
|    0|shocking new movi...|
|    1|about 500 french ...|
|    0| state department...|
|    0|john mccain throw...|
|    0|forbes list of “t...|
|    1|hurricane irma th...|
|    0|was iranian busin...|
|    0|from the religion...|
|    1|pakistan bars a m...|
|    0|gop presidential ...|
|    0|ep #10: patrick h...|
|    0|judiciary member:...|
|    0|‘you’re a sexist ...|
+-----+--------------------+
only showing top 20 rows



In [66]:
news_data_df.groupBy(col('label')).count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0|23481|
|    1|21417|
+-----+-----+



In [67]:
#Conver PySpark to Pandas
news_data_pd_df = news_data_df.toPandas()
news_data_pd_df.head()

,label,news
0,0,meet leftist vogue fashion editor slammed on s...
1,1,netanyahu muzzles israeli officials on kurdish...
2,1,china and india are development opportunities ...
3,0,hell-bent on a conviction: is the pentagon’s t...
4,0,"the new sweden: rapes, riots, severed heads, f..."


In [68]:
# Remove urls
def urls_remove(text):
    pattern_url = re.compile(r'(https?://\S+)|(www\.\S+)|(\S+\.\S+/\S+)')
    return pattern_url.sub(r'', text)

news_data_pd_df['urlsRemoved'] = news_data_pd_df['news'].apply(urls_remove)

news_data_pd_df

,label,news,urlsRemoved
0,0,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...
1,1,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...
2,1,china and india are development opportunities ...,china and india are development opportunities ...
3,0,hell-bent on a conviction: is the pentagon’s t...,hell-bent on a conviction: is the pentagon’s t...
4,0,"the new sweden: rapes, riots, severed heads, f...","the new sweden: rapes, riots, severed heads, f..."
...,...,...,...
44893,1,"obama, republican kasich meet to strategize on...","obama, republican kasich meet to strategize on..."
44894,0,this is not a joke! soros-linked group has pla...,this is not a joke! soros-linked group has pla...
44895,1,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...
44896,1,u.n. tells north korea envoy that pyongyang mu...,u.n. tells north korea envoy that pyongyang mu...


In [69]:
#Remove all special characters.
def remove_special_chars(text):
    pattern = r'[^\w\s]' #([^\w\s] matches any character that is not a word character (\w) or whitespace (\s), which includes emoticons, symbols & pictographs, transport & map symbols, flags (iOS), other miscellaneous symbols, enclosed characters.)
    return re.sub(pattern, '', text)
news_data_pd_df['specialcharRemoved'] = news_data_pd_df['urlsRemoved'].apply(remove_special_chars)
news_data_pd_df

,label,news,urlsRemoved,specialcharRemoved
0,0,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...
1,1,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...
2,1,china and india are development opportunities ...,china and india are development opportunities ...,china and india are development opportunities ...
3,0,hell-bent on a conviction: is the pentagon’s t...,hell-bent on a conviction: is the pentagon’s t...,hellbent on a conviction is the pentagons thir...
4,0,"the new sweden: rapes, riots, severed heads, f...","the new sweden: rapes, riots, severed heads, f...",the new sweden rapes riots severed heads freel...
...,...,...,...,...
44893,1,"obama, republican kasich meet to strategize on...","obama, republican kasich meet to strategize on...",obama republican kasich meet to strategize on ...
44894,0,this is not a joke! soros-linked group has pla...,this is not a joke! soros-linked group has pla...,this is not a joke soroslinked group has plan ...
44895,1,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...
44896,1,u.n. tells north korea envoy that pyongyang mu...,u.n. tells north korea envoy that pyongyang mu...,un tells north korea envoy that pyongyang must...


In [70]:
# Remove all stop word
nltk.download('stopwords')
nltk.download('punkt')

def stopword(text):
   stop_words = set(stopwords.words('english'))
   return " ".join([word for word in text.split() if word not in stop_words])

news_data_pd_df['stopWordRemoved'] = news_data_pd_df['specialcharRemoved'].apply(stopword)
news_data_pd_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,label,news,urlsRemoved,specialcharRemoved,stopWordRemoved
0,0,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed soci...
1,1,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials kurdish re...
2,1,china and india are development opportunities ...,china and india are development opportunities ...,china and india are development opportunities ...,china india development opportunities threats ...
3,0,hell-bent on a conviction: is the pentagon’s t...,hell-bent on a conviction: is the pentagon’s t...,hellbent on a conviction is the pentagons thir...,hellbent conviction pentagons third attempt co...
4,0,"the new sweden: rapes, riots, severed heads, f...","the new sweden: rapes, riots, severed heads, f...",the new sweden rapes riots severed heads freel...,new sweden rapes riots severed heads freeloadi...
...,...,...,...,...,...
44893,1,"obama, republican kasich meet to strategize on...","obama, republican kasich meet to strategize on...",obama republican kasich meet to strategize on ...,obama republican kasich meet strategize tpp tr...
44894,0,this is not a joke! soros-linked group has pla...,this is not a joke! soros-linked group has pla...,this is not a joke soroslinked group has plan ...,joke soroslinked group plan destroy trumpwill ...
44895,1,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying buy items missile developmen...
44896,1,u.n. tells north korea envoy that pyongyang mu...,u.n. tells north korea envoy that pyongyang mu...,un tells north korea envoy that pyongyang must...,un tells north korea envoy pyongyang must stop...


In [71]:
from nltk.stem.wordnet import WordNetLemmatizer
# Download the wordnet resource
nltk.download('wordnet')

# Define the lemmatization function
def lemmatize_text(text):
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])

news_data_pd_df['lemmatized_Rows'] = news_data_pd_df['stopWordRemoved'].apply(lemmatize_text)

news_data_pd_df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,label,news,urlsRemoved,specialcharRemoved,stopWordRemoved,lemmatized_Rows
0,0,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed soci...,meet leftist vogue fashion editor slammed soci...
1,1,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials kurdish re...,netanyahu muzzle israeli official kurdish refe...
2,1,china and india are development opportunities ...,china and india are development opportunities ...,china and india are development opportunities ...,china india development opportunities threats ...,china india development opportunity threat xi ...
3,0,hell-bent on a conviction: is the pentagon’s t...,hell-bent on a conviction: is the pentagon’s t...,hellbent on a conviction is the pentagons thir...,hellbent conviction pentagons third attempt co...,hellbent conviction pentagon third attempt con...
4,0,"the new sweden: rapes, riots, severed heads, f...","the new sweden: rapes, riots, severed heads, f...",the new sweden rapes riots severed heads freel...,new sweden rapes riots severed heads freeloadi...,new sweden rape riot severed head freeloading ...
...,...,...,...,...,...,...
44893,1,"obama, republican kasich meet to strategize on...","obama, republican kasich meet to strategize on...",obama republican kasich meet to strategize on ...,obama republican kasich meet strategize tpp tr...,obama republican kasich meet strategize tpp tr...
44894,0,this is not a joke! soros-linked group has pla...,this is not a joke! soros-linked group has pla...,this is not a joke soroslinked group has plan ...,joke soroslinked group plan destroy trumpwill ...,joke soroslinked group plan destroy trumpwill ...
44895,1,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying buy items missile developmen...,iran still trying buy item missile development...
44896,1,u.n. tells north korea envoy that pyongyang mu...,u.n. tells north korea envoy that pyongyang mu...,un tells north korea envoy that pyongyang must...,un tells north korea envoy pyongyang must stop...,un tell north korea envoy pyongyang must stop ...


In [72]:
#Remove all tokens
def remove_tokens(text):
    # Remove repeating characters
    text = re.sub(r'(\w)\1+', r'\1', text)

    # Remove tokens containing digits
    text = re.sub(r'\b\w*\d\w*\b', '', text)

    # Remove tokens containing underscore
    text = re.sub(r'\b\w*_\w*\b', '', text)

    # Remove tokens containing special characters
    text = re.sub(r'\b\w*[^\w\s]+\w*\b', '', text)

    # Remove tokens less than 2 characters
    text = re.sub(r'\b\w{1}\b', '', text)

    return text

news_data_pd_df['final_news'] = news_data_pd_df['lemmatized_Rows'].apply(remove_tokens)

news_data_pd_df

,label,news,urlsRemoved,specialcharRemoved,stopWordRemoved,lemmatized_Rows,final_news
0,0,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed on s...,meet leftist vogue fashion editor slammed soci...,meet leftist vogue fashion editor slammed soci...,met leftist vogue fashion editor slamed social...
1,1,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials on kurdish...,netanyahu muzzles israeli officials kurdish re...,netanyahu muzzle israeli official kurdish refe...,netanyahu muzle israeli oficial kurdish refere...
2,1,china and india are development opportunities ...,china and india are development opportunities ...,china and india are development opportunities ...,china india development opportunities threats ...,china india development opportunity threat xi ...,china india development oportunity threat xi t...
3,0,hell-bent on a conviction: is the pentagon’s t...,hell-bent on a conviction: is the pentagon’s t...,hellbent on a conviction is the pentagons thir...,hellbent conviction pentagons third attempt co...,hellbent conviction pentagon third attempt con...,helbent conviction pentagon third atempt convi...
4,0,"the new sweden: rapes, riots, severed heads, f...","the new sweden: rapes, riots, severed heads, f...",the new sweden rapes riots severed heads freel...,new sweden rapes riots severed heads freeloadi...,new sweden rape riot severed head freeloading ...,new sweden rape riot severed head freloading f...
...,...,...,...,...,...,...,...
44893,1,"obama, republican kasich meet to strategize on...","obama, republican kasich meet to strategize on...",obama republican kasich meet to strategize on ...,obama republican kasich meet strategize tpp tr...,obama republican kasich meet strategize tpp tr...,obama republican kasich met strategize tp trad...
44894,0,this is not a joke! soros-linked group has pla...,this is not a joke! soros-linked group has pla...,this is not a joke soroslinked group has plan ...,joke soroslinked group plan destroy trumpwill ...,joke soroslinked group plan destroy trumpwill ...,joke soroslinked group plan destroy trumpwil r...
44895,1,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying to buy items for missile dev...,iran still trying buy items missile developmen...,iran still trying buy item missile development...,iran stil trying buy item misile development g...
44896,1,u.n. tells north korea envoy that pyongyang mu...,u.n. tells north korea envoy that pyongyang mu...,un tells north korea envoy that pyongyang must...,un tells north korea envoy pyongyang must stop...,un tell north korea envoy pyongyang must stop ...,un tel north korea envoy pyongyang must stop d...


In [73]:
#Remove extra columns.
clean_news_data_pd_df = news_data_pd_df.drop(['news', 'urlsRemoved', 'specialcharRemoved',  'stopWordRemoved', 'lemmatized_Rows'],axis=1)

clean_news_data_pd_df

,label,final_news
0,0,met leftist vogue fashion editor slamed social...
1,1,netanyahu muzle israeli oficial kurdish refere...
2,1,china india development oportunity threat xi t...
3,0,helbent conviction pentagon third atempt convi...
4,0,new sweden rape riot severed head freloading f...
...,...,...
44893,1,obama republican kasich met strategize tp trad...
44894,0,joke soroslinked group plan destroy trumpwil r...
44895,1,iran stil trying buy item misile development g...
44896,1,un tel north korea envoy pyongyang must stop d...


In [74]:
clean_news_data_pd_df.dtypes

label          int32
final_news    object
dtype: object

# Vectorization (Convert Text data into the Vector)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(clean_news_data_pd_df['final_news'], clean_news_data_pd_df['label'], random_state=0)

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Use TfidfVectorizer to add weight to repeated words
Vectorizer = TfidfVectorizer(max_features=100)
x = Vectorizer.fit_transform(clean_news_data_pd_df['final_news']).toarray()
feature_names = Vectorizer.get_feature_names_out()

In [77]:
y = clean_news_data_pd_df['label']
y.head()

0    0
1    1
2    1
3    0
4    0
Name: label, dtype: int32

In [78]:
feature_names[:1000]

array(['acording', 'administration', 'also', 'america', 'american',
       'atack', 'back', 'bil', 'campaign', 'city', 'clinton', 'could',
       'country', 'court', 'day', 'democrat', 'democratic', 'department',
       'donald', 'election', 'even', 'federal', 'first', 'foreign',
       'former', 'get', 'go', 'going', 'government', 'group', 'hilary',
       'house', 'image', 'including', 'know', 'last', 'law', 'leader',
       'like', 'made', 'make', 'many', 'may', 'medium', 'member',
       'milion', 'military', 'month', 'national', 'ned', 'new', 'news',
       'north', 'obama', 'ofice', 'oficial', 'one', 'party', 'people',
       'percent', 'plan', 'police', 'policy', 'political', 'president',
       'presidential', 'public', 'report', 'republican', 'reuters',
       'right', 'rusia', 'rusian', 'said', 'say', 'security', 'senate',
       'since', 'state', 'statement', 'suport', 'take', 'tax', 'think',
       'time', 'told', 'trump', 'two', 'united', 'video', 'vote', 'want',
       'w

In [79]:
x

array([[0.        , 0.        , 0.07620419, ..., 0.        , 0.13578787,
        0.        ],
       [0.        , 0.        , 0.18786436, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.17827989, 0.        , 0.12652031, ..., 0.        , 0.        ,
        0.26170692],
       [0.        , 0.        , 0.        , ..., 0.20816393, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08387714, 0.10119708,
        0.05873687]])

In [80]:
x=pd.DataFrame(x, columns = feature_names)

In [81]:
x.shape

(44898, 100)

# Data splitting into the train and test

In [82]:
#Split Data using train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 10, stratify = y )

In [83]:
 x_train.shape

(31428, 100)

In [84]:
 x_test.shape

(13470, 100)

In [85]:
import sklearn as skl
# Create scaler instance
x_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
x_scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [86]:
x_train_scaled.shape

(31428, 100)

In [87]:
from keras.layers import Embedding

# Define the model - LSTM neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Add the embedding layer
nn.add(Embedding(input_dim=1000, output_dim=64, input_length=x_train_scaled.shape[1]))

# First hidden layer
nn.add(tf.keras.layers.LSTM(units=80, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           64000     
                                                                 
 lstm_1 (LSTM)               (None, 80)                46400     
                                                                 
 dense_2 (Dense)             (None, 30)                2430      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 112861 (440.86 KB)
Trainable params: 112861 (440.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [88]:
 # Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer= "adam", metrics=['accuracy'])

# Fit the model to the training data
fit_model = nn.fit(x_train_scaled, y_train,epochs=10)

Epoch 1/10
983/983 [==============================] - 137s 136ms/step - loss: 33.5425 - accuracy: 0.5522
Epoch 2/10
983/983 [==============================] - 134s 136ms/step - loss: 17100335742976.0000 - accuracy: 0.5608
Epoch 3/10
983/983 [==============================] - 133s 135ms/step - loss: 0.6829 - accuracy: 0.5598
Epoch 4/10
983/983 [==============================] - 133s 136ms/step - loss: 0.6812 - accuracy: 0.5611
Epoch 5/10
983/983 [==============================] - 133s 135ms/step - loss: 0.6809 - accuracy: 0.5615
Epoch 6/10
983/983 [==============================] - 133s 136ms/step - loss: 0.6806 - accuracy: 0.5634
Epoch 7/10
983/983 [==============================] - 134s 137ms/step - loss: 0.6804 - accuracy: 0.5610
Epoch 8/10
983/983 [==============================] - 136s 138ms/step - loss: 0.6802 - accuracy: 0.5642
Epoch 9/10
983/983 [==============================] - 134s 136ms/step - loss: 0.6802 - accuracy: 0.5629
Epoch 10/10
983/983 [=============================

In [89]:
 # Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

421/421 - 8s - loss: 0.6811 - accuracy: 0.5623 - 8s/epoch - 18ms/step
Loss: 0.6811049580574036, Accuracy: 0.5622865557670593
